In [9]:
library(Matrix)
library(Seurat)

Attaching SeuratObject



In [17]:
counts <- readMM('../../data/fucci/matrix.mtx')
rownames(counts) <- read.csv('../../data/fucci/features.tsv')$x
colnames(counts) <- read.csv('../../data/fucci/barcodes.tsv')$x
meta <- read.csv('../../data/fucci/meta.csv', row.names=1)

In [18]:
obj <- CreateSeuratObject(counts, meta.data=meta)
obj@meta.data$sample <- as.logical(sample(c(TRUE,FALSE), ncol(obj), replace = TRUE))

obj_orig <- subset(obj, subset=(sample == TRUE))
obj_to_ds <- subset(obj, subset=(sample != TRUE))
expr_ds <- apply(obj_to_ds$RNA@counts, c(1,2), function(n){rbinom(1,size=n, prob=0.5)})
obj_ds <- CreateSeuratObject(expr_ds, meta.data=obj_to_ds@meta.data[,4:ncol(obj_to_ds@meta.data)])

obj_orig@meta.data$label <- 'Original'
obj_ds@meta.data$label <- 'Downsampled'
obj_merge <- merge(obj_orig, y=obj_ds)

In [20]:
df <- data.frame('x1'=cos(obj_merge@meta.data$theta), 
                 'x2'=sin(obj_merge@meta.data$theta), 
                 'y'=log10(obj_merge@meta.data$nCount_RNA), 
                 'label'=obj_merge@meta.data$label, 
                 'x4'=obj_merge@meta.data$S.Score,
                 'x5'=obj_merge@meta.data$G2M.Score
                )
df$x3 <- df$x1-df$x2

In [23]:
set.seed(777)

qs <- quantile(df$y, (0:30)/100)
lst1 <- lapply(qs, function(q){
    summary(lm('x3~factor(label)', data=df[df$y>q,]))$coefficients[2,'Pr(>|t|)']
})
lst2 <- lapply(qs, function(q){
    summary(lm('x5~factor(label)', data=df[df$y>q,]))$coefficients[2,'Pr(>|t|)']
})
df_result_2 <- data.frame('Fucci score'=unlist(lst1), 'Seurat score'=unlist(lst2))
write.csv(df_result_2, 'fig1h.csv')